# Pipeline generators

This demonstrates how Pipeline generation can occur even with native (unmodified) `intbv` types.

In [1]:
import sys
sys.path.insert(0, '../..')
from myirl import *
from myhdl import intbv

d = DummyVHDLModule()

First, derive a PipelineProcess class from `Process`:

In [2]:
# @hls.generator
class PipelineProcess(kernel.sensitivity.Process):
    def __init__(self, func, clk, logic, stage):
        def f():
            print("FUNC RETURN LOGIC", logic)
            return logic
    
        self.func = f
        f.__name__ = func.__name__ + "_stage%d" % stage
        self.sensors = [clk]
        self.edge = clk.POS
        self.reset = None
        self.logic = LogicContext()
        self.logic += f()
    # Note: This should never be called, so we don't need a __call__ method


Then implement a Pipeline class for the `@pipeline` decorator magic:

In [3]:
class Pipeline(Generator):
    def __init__(self, func, clk, queue):
        self.clk = clk
        self.queue = queue
        super().__init__(func)

    def __call__(self, ctx):
        for i, g in enumerate(self.func()):
            self.logic += [
                PipelineProcess(self.func, self.clk, g, i)
            ]
        # Add a local reset signal:
        lreset = ResetSignal(ResetSignal.NEG_SYNC)
        
        @genprocess(self.clk, EDGE=self.clk.POS, RESET=lreset)
        def delay_queue():
            en = self.queue
            N = len(en)
            for i in range(1, N):
                yield [
                    en[i].set(en[i-1])            
                ]

        # Important to call that process within the
        # actual context:
        delay_queue(ctx)

        self.logic += [
            delay_queue,
            lreset.wireup(self.queue[0])
        ]
            
    def collect_sources(self):
        signals = {}
        for i in self.logic:
            signals.update(i.collect_sources())
        
        return signals

    def collect_drivers(self):
        signals = {}
        for i in self.logic:
            signals.update(i.collect_drivers())
        
        return signals    
    
def pipeline(clk, queue, *kwargs):
    def pipeline_dec(func):
        return Pipeline(func, clk, queue)

    return pipeline_dec   

Now create a pipeline test:

In [4]:
@block
def dummy(clk: ClkSignal, en : Signal, din : Signal,
          dout: Signal.Output, valid : Signal.Output):
    a = Signal(intbv()[7:], name = 'a')
    
    enable = [ Signal(bool(), name = 'en%d' % i) for i in range(4)]

    q = Signal(intbv()[8:], name = 'q')
    p = Signal(intbv()[5:], name = 'p')

    @pipeline(clk, enable)
    def pipe():
        v = Variable('v', intbv(5)[7:])
        # First stage
        yield [
            v.assign(din * base.ConstSig(2, 2)),
            a.set(v),
        ]
        
        # Second stage
        yield [
            q.set(a + 4),
        ]
        
        # Third stage
        yield [
            p.set(q[6:1]), # value >> 1
        ]
        
    wires = [
        enable[0].wireup(en),
        dout.wireup(p),
        valid.wireup(enable[2])
    ]
        
    return locals()

from myirl.test.common_test import *
from myirl.simulation import *

@block
def tb():
    clk = ClkSignal(name = 'clk')
    en, valid = [ Signal(bool()) for _ in range(2) ]
    data_in = Signal(intbv()[5:])
    data_out = Signal(intbv()[5:])
    
    inst = dummy(clk, en, data_in, data_out, valid)
 
    osc = gen_osc(clk, CYCLE=5)
    
    @generator
    def seq():
        yield [ en.set(False) ]
        for i in range(4):
            yield [ wait(clk.posedge) ]
            
        yield [
            data_in.set(0xa),
            wait(clk.posedge),
            en.set(True),
            
        ]
            

        
        yield [ wait(clk.posedge, ) for i in range(4) ]
        
        yield [
            print_("data out:", data_out),
            raise_(StopSimulation)
        ]
        yield [ wait(clk.posedge, ) for i in range(4) ]

    return locals()
 
def test():
    inst = tb()
    files = inst.elab(targets.VHDL, elab_all = True)
    run_ghdl(files, inst, debug = True, vcdfile='/tmp/tb_pipe.vcd')
    
test()

Creating generator '__init__/pipe' 
 Insert unit dummy_s1_s1_s5_s5_s1 
Creating delay 'gen_osc/CLKGEN' : 5 ns
Creating sequential 'tb/seq' 
 Insert unit tb 
FUNC RETURN LOGIC [v = MUL(din, <C: 0x2:2>), a <= <v:05>]
FUNC RETURN LOGIC [q <= ADD_builtin(a, C:4)]
FUNC RETURN LOGIC [p <= <q[6:1]>]
Creating Generator process '__call__/delay_queue' with sensitivity (<clk>,)
DEBUG CALL GENERATOR


../../myirl/kernel/components.py:789: UserWarning: Already collected en0
  base.warnings.warn("Already collected %s" % n)
../../myirl/kernel/components.py:789: UserWarning: Already collected clk
  base.warnings.warn("Already collected %s" % n)


 DEBUG: Writing 'dummy' to file /tmp/dummy.vhdl 
Finished _elab in 0.0014 secs


../../myirl/kernel/components.py:789: UserWarning: Already collected en
  base.warnings.warn("Already collected %s" % n)
../../myirl/kernel/components.py:789: UserWarning: Already collected data_in
  base.warnings.warn("Already collected %s" % n)


 DEBUG: Writing 'tb' to file /tmp/tb.vhdl 
Finished _elab in 0.0015 secs
==== COSIM stdout ====

==== COSIM stderr ====

==== COSIM stdout ====
analyze ../../myirl/targets/../test/vhdl/txt_util.vhdl
analyze ../../myirl/targets/libmyirl.vhdl
analyze /tmp/dummy.vhdl
analyze /tmp/tb.vhdl
elaborate tb

==== COSIM stderr ====

==== COSIM stdout ====
data out: 0x0C
/tmp/tb.vhdl:56:9:@85ns:(assertion failure): Stop Simulation
./tb:error: assertion failed
in process .tb(myirl).seq
./tb:error: simulation failed

==== COSIM stderr ====



**Note again**: Incorrect wave display for the 'tb.valid'. GTKwave will display correctly.

In [18]:
import wavedraw; import nbwavedrom
TB = tb.name;
cfg = {
    'tb.clk' : None,
    'tb.valid' : None, 'tb.data_out[4:0]' : None, 'tb.data_in[4:0]' : None, 
    'tb.inst_dummy_0.q[7:0]' : None, 'tb.inst_dummy_0.a[6:0]' : None,
}
waveform = wavedraw.vcd2wave("/tmp/tb_pipe.vcd", TB + '.clk', cfg)
nbwavedrom.draw(waveform)